<a href="https://colab.research.google.com/github/niinab/Ohjelmistokehityksentekn_seminaarityo/blob/main/Seminaarity%C3%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import datetime, os
import numpy as np
%load_ext tensorboard




In [ ]:
# Datasetin lataus

mnist = tf.keras.datasets.fashion_mnist

In [ ]:
# Training ja test-versioiden lataus
# Labels (0 T-shirt/top, 1 Trouser, 2 Pullover, 3 Dress, 4 Coat, 5 Sandals, 6 Shirt, 7 Sneaker, 8 Bag, 9 Ankle boot)

(training_images, training_labels),(test_images, test_labels) = mnist.load_data()

In [ ]:
import matplotlib.pyplot as plt
# Ladataan testiversio datasta, (tässä otettu item nro 6)

plt.imshow(training_images[6])
print(training_labels[6])
print(training_images[6])

In [ ]:
# Kuvadata on esitetty lukuina 0-255 (pixels). Normalisoidaan kuvat jakamalla datamäärä luvulla 255

training_images, test_images = training_images / 255.0, test_images / 255.0

In [ ]:
# Varmistetaan, missä minkä kokoisia kuvat ovat. Komento tulostaa myös kuvien määrän

training_images.shape

In [ ]:
# Testidatan kuvia on 10000, ja 28x28 pixeliä  

test_images.shape

In [ ]:
# Määritellään malli neljällä eri tasolla. 
# Sequential > määrittelee tasojen järjestyksen neuroverkossa
# Flatten > muuttaa neliön yksiulotteiseksi vektoriksi
# Dense > lisää neuronitason
# Activation kertoo jokaiselle neuronikerrokselle mitä tehdä
# Relu > if x > 0 return X, else return 0. Palauttaa arvon 0 tai suurempi seuraavalle verkkokerrokselle
# Dropout > Jättää satunnaisella otannalla tietyn osan neuroverkosta käsittelemättä. Estää ylisovitusta. 
# Softmax > Neuroverkon viimeinen ulostulotaso. Käytännössä suorittaa joukon arvoja ja palauttaa niistä suurimman

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)), 
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
# Mallin määrittelyn jälkeen tulee kouluttamisvaihe 
# Metrics "accuracy" raportoi kouluttamisen tarkkuuden vertaamalla tulosta tiedossa oleviin nimikkeisiin(labels)
# Tarkoituksena on, että malli ymmärtää suhteen traning kuvien ja training labelsejen välillä
# Validation datalla nähdään miten malli käyttäytyy tuntemattoman datan kanssa (test data)

model.compile(optimizer ='Adam',
              loss ='sparse_categorical_crossentropy',
              metrics=['accuracy'])

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(x=training_images, y=training_labels, epochs=7, validation_data=(test_images, test_labels), verbose=2, callbacks=[tensorboard_callback])

In [ ]:
tensorboard --logdir logs

In [ ]:
# Callback-funktiolla voidaan määritellä max tarkkuusrajan 
# Alla olevassa esimerkissä rajana on 90%, jonka jälkeen training pysäytetään vaikka epocheja olisi vielä jäljellä.

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

In [ ]:
#Tulostetaan mallin yhteenveto

model.summary()

In [ ]:
# Mallin voi tulostaa JSON-muotoon

json_string = model.to_json()

json_string

In [ ]:
# Malli voidaan uudelleen rakentaa JSON-tiedostosta. 

from tensorflow.keras.models import model_from_json
model_architecture = model_from_json(json_string)

# Mallin arkkitehtuurinen yhteenveto. Huom. ei sisällä koulutettua dataa, loss tai accuracy tietoja. 
model_architecture.summary()

Predictions

In [ ]:
predictions = model.predict(test_images, verbose=0)


In [ ]:
predictions[0]

In [ ]:
plt.figure()
plt.imshow(test_images[0])
plt.colorbar()
plt.show()

Confusion Matrix

In [ ]:
# %matplotlib inline
# from sklearn.metrics import confusion_matrix
# import itertools
# import matplotlib.pyplot as plt



In [ ]:
# cm = confusion_matrix(y_true=test_labels, y_pred=rounded_predictions)

In [ ]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
    
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=90)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

In [ ]:
# # Predict the values from the validation dataset
# Y_pred = model.predict(test_labels)
# # Convert predictions classes to one hot vectors 
# Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# # Convert validation observations to one hot vectors
# Y_true = np.argmax(test_images,axis = 1)
# # compute the confusion matrix
# confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# # plot the confusion matrix
# plot_confusion_matrix(confusion_mtx, 
#             classes = ['T-shirt/Top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot'])